In [2]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import *

Sample dummy submission:

In [114]:
train = pd.read_csv('train_data.csv')
test = pd.read_csv('test_data.csv')

File with answers:

In [115]:
y_test = pd.read_csv('test_data_labels.csv')

In [117]:
y_test.sample()

,comment_id,toxic
2930,2930,0


In [129]:
y_test = list(y_test['toxic'])

In [109]:
sample_submission = [[i, randint(0,1)] for i in range(len(test))]
sample_submission = pd.DataFrame(sample_submission, columns=['comment_id', 'toxic'])

sample_submission.head()

,comment_id,toxic
0,0,0
1,1,1
2,2,0
3,3,1
4,4,1


In [84]:
sample_submission.to_csv('sample_submission.csv', index=False)

In [112]:
train['toxic'] = train.toxic.astype(int)

In [118]:
y_train = train['toxic']

##  baseline 1: no preprocessing + bow -> 4

In [73]:
from sklearn.linear_model import LogisticRegression 
from sklearn.feature_extraction.text import CountVectorizer

In [65]:
text = ['april, boo boo ceph', 'demon - april ceph i']

In [50]:
print('\\6')

\6


In [91]:
vec = CountVectorizer(ngram_range=(1, 1), token_pattern='\w{3,}') # строим BoW для слов

In [76]:
#help(CountVectorizer)

In [67]:
vec.fit(text)

CountVectorizer(token_pattern='\\w+')

In [68]:
vec.vocabulary_.keys()

dict_keys(['april', 'boo', 'ceph', 'demon', 'i'])

In [69]:
len(vec.vocabulary_.keys())

5

In [70]:
bow = vec.transform(text)
bow

<2x5 sparse matrix of type '<class 'numpy.int64'>'
	with 7 stored elements in Compressed Sparse Row format>

In [71]:
print(text)

['april, boo boo ceph', 'demon - april ceph i']


In [72]:
bow.toarray()

array([[1, 2, 1, 0, 0],
       [1, 0, 1, 1, 1]])

In [10]:
vec.ngram_range

(1, 1)

In [8]:
#help(CountVectorizer)

In [74]:
train

,comment,toxic
0,Преступление и наказание\n,0.0
1,"И именно эти неработающие весы показывают, что...",0.0
2,"В Японии такие панельки, ебанько.\n",0.0
3,Еще у нас выявляют трещины с помощью белой кра...,0.0
4,"Дочитал до поезда в Норильск , дальше не стал\n",0.0
...,...,...
10804,А у мамы в группе до самого выпуска из сада та...,0.0
10805,Сука тупой дегенарт. Вот на эти видео. Съеби у...,1.0
10806,"В племенах украинцев, особенно западных, с дет...",1.0
10807,"Пост про жадность, о том, как человек оплативш...",0.0


In [ ]:
C

In [92]:
bow = vec.fit_transform(train['comment'])

In [93]:
bow

<10809x55942 sparse matrix of type '<class 'numpy.int64'>'
	with 210108 stored elements in Compressed Sparse Row format>

In [84]:
print(train.comment[10804])

А у мамы в группе до самого выпуска из сада такие просяки нет-нет, да случаются, с разными детьми. Только для одних детей это разовая акция, у других - время от времени, а у третьих - частенько. И родители прекрасно в курсе особенностей каждого конкретного ребенка.



In [79]:
list(vec.vocabulary_.items())[:10]

[('преступление', 37597),
 ('и', 16431),
 ('наказание', 25325),
 ('именно', 16939),
 ('эти', 56202),
 ('неработающие', 27074),
 ('весы', 6263),
 ('показывают', 35049),
 ('что', 54896),
 ('работающих', 40452)]

In [88]:
sorted(list(vec.vocabulary_.items()), key=lambda x: x[0])[:10]

[('0', 0),
 ('00', 1),
 ('000', 2),
 ('0015', 3),
 ('003', 4),
 ('0036', 5),
 ('005', 6),
 ('00х', 7),
 ('01', 8),
 ('013', 9)]

In [87]:
#list(vec.vocabulary_.keys())

In [80]:
len(vec.vocabulary_.items())

56619

In [97]:
y_train = train['toxic'].astype(int)
y_train

0        0
1        0
2        0
3        0
4        0
        ..
10804    0
10805    1
10806    1
10807    0
10808    0
Name: toxic, Length: 10809, dtype: int64

In [101]:
#help(clf.fit)

In [102]:
clf = LogisticRegression(random_state=42, max_iter=500) #
clf.fit(bow, y_train)

LogisticRegression(max_iter=500, random_state=42)

In [105]:
len(clf.coef_[0])

55942

In [107]:
bow_test = vec.transform(test['comment'])
bow_test

<3603x55942 sparse matrix of type '<class 'numpy.int64'>'
	with 59233 stored elements in Compressed Sparse Row format>

In [109]:
pred = clf.predict(bow_test)
pred[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 1, 0])

In [112]:
test

,comment_id,comment
0,0,"В данном случае не пионервожатая, а техничка, ..."
1,1,В данном посте рассматривается обычный ленточн...
2,2,"Как я понимаю, у поверхностей отражаемость пог..."
3,3,Австралийские пауки съедят все живое на планете))
4,4,У нас раньше Амур пиво целые бутылки принимал ...
...,...,...
3598,3598,"Я всё жду, когда эта херня закончится\n"
3599,3599,Откуда такой акцент на Москве? У нас между про...
3600,3600,"Кого угодно, но не этих двух отбросов: свинью ..."
3601,3601,А как же радиоуправляемые машинки в клубах?\n


In [116]:
print(classification_report(pred, y_test['toxic']))

              precision    recall  f1-score   support

           0       0.94      0.84      0.89      2651
           1       0.66      0.84      0.74       952

    accuracy                           0.84      3603
   macro avg       0.80      0.84      0.81      3603
weighted avg       0.86      0.84      0.85      3603



In [119]:
from sklearn.metrics import accuracy_score

accuracy_score(pred, y_test['toxic'])

0.8434637801831807

In [90]:
baseline_4 = [[i, pred[i]] for i in range(len(test))]
baseline_4 = pd.DataFrame(baseline_4, columns=['comment_id', 'toxic'])

baseline_4.head()

,comment_id,toxic
0,0,0
1,1,0
2,2,0
3,3,0
4,4,0


In [92]:
baseline_4.to_csv('baseline_4.csv', index=False)

## baseline 2: preprocessing + bow -> 5

In [122]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [121]:
!pip install pymorphy2

     |████████████████████████████████| 55 kB 1.2 MB/s eta 0:00:01
     |████████████████████████████████| 8.2 MB 5.8 MB/s eta 0:00:01
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13724 sha256=191796ef4b5a74e501f5d0d99fb079755515fa7c82ce263bc2a5c7b564ee0e3f
  Stored in directory: /Users/veronica/Library/Caches/pip/wheels/70/4a/46/1309fc853b8d395e60bafaf1b6df7845bdd82c95fd59dd8d2b
Successfully built docopt


In [124]:
import re
from pymorphy2 import MorphAnalyzer
from functools import lru_cache
from nltk.corpus import stopwords

m = MorphAnalyzer()
regex = re.compile("[а-яa-zёЁ]+")

def words_only(text, regex=regex):
    try:
        return regex.findall(text.lower())
    except:
        return []

In [127]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [126]:
words_only(train.comment[1])

['и',
 'именно',
 'эти',
 'неработающие',
 'весы',
 'показывают',
 'что',
 'работающих',
 'нет']

In [130]:
%time lemmatize_word('неработающие')

CPU times: user 529 µs, sys: 187 µs, total: 716 µs
Wall time: 1.48 ms


'неработающий'

In [136]:
%time lemmatize_word('неработающие')

CPU times: user 5 µs, sys: 1 µs, total: 6 µs
Wall time: 11 µs


'неработающий'

In [138]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/veronica/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [139]:
@lru_cache(maxsize=128)
def lemmatize_word(token, pymorphy=m):
    return pymorphy.parse(token)[0].normal_form

def lemmatize_text(text):
    return [lemmatize_word(w) for w in text]


mystopwords = stopwords.words('russian') 
def remove_stopwords(lemmas, stopwords = mystopwords):
    return [w for w in lemmas if not w in stopwords and len(w) > 3]

def clean_text(text):
    tokens = words_only(text)
    lemmas = lemmatize_text(tokens)
    
    return ' '.join(remove_stopwords(lemmas))

In [141]:
train.comment[1]

'И именно эти неработающие весы показывают, что работающих нет?..\n'

In [140]:
clean_text(train.comment[1])

'именно неработающий весы показывать работать'

In [142]:
from tqdm import tqdm

lemmas = list(tqdm(map(clean_text, train['comment']), total=len(train)))
    
train['lemmas'] = lemmas
train.sample(5)

100%|████████████████████████████████████| 10809/10809 [00:24<00:00, 437.87it/s]


,comment,toxic,lemmas
5946,ну так лицензией это никогда и не было. Неудив...,0.0,лицензия неудивительный активироваться
7623,Гасят звуковую волну еще и разной толщины мате...,0.0,гасить звуковой волна разный толщина материал ...
9977,"Не, игра Ballance круче! Хотел было найти гифк...",0.0,игра ballance крутой хотеть найти гифка падени...
8030,Укосины в сответствии с американскими codes вы...,0.0,укосина сответствие американский codes выполен...
1267,"Проблема ещё и в том, что ей нужно паспорт мен...",0.0,проблема нужно паспорт менять возраст говорить...


In [143]:
lemmas_test = list(tqdm(map(clean_text, test['comment']), total=len(test)))
    
test['lemmas'] = lemmas_test

100%|██████████████████████████████████████| 3603/3603 [00:08<00:00, 419.76it/s]


In [148]:
vec = TfidfVectorizer(ngram_range=(1, 2)) # строим BoW для слов
bow = vec.fit_transform(train['lemmas'])

clf = LogisticRegression(random_state=42, max_iter=500)
clf.fit(bow, y_train)

pred = clf.predict(vec.transform(test['lemmas']))
accuracy_score(pred, y_test['toxic'])

0.8245906189286706

In [95]:
baseline_5 = [[i, pred[i]] for i in range(len(test))]
baseline_5 = pd.DataFrame(baseline_5, columns=['comment_id', 'toxic'])

baseline_5.to_csv('baseline_5.csv', index=False)

## baseline 3: preproc + fasttext -> 6

In [34]:
import fasttext

In [39]:
with open('train_ft.txt', 'w') as f:
    for pair in list(zip(train['lemmas'], train['toxic'])):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [41]:
with open('test_ft.txt', 'w') as f:
    for pair in list(zip(test['lemmas'], test['toxic'])):
        text, label = pair
        f.write(f'__label__{label} {text.lower()}\n')

In [99]:
classifier = fasttext.train_supervised('train_ft.txt')#, 'model')
result = classifier.test('test_ft.txt')
print('P@1:', result[1])#.precision)
print('R@1:', result[2])#.recall)
print('Number of examples:', result[0])#.nexamples)

P@1: 0.8739938939772411
R@1: 0.8739938939772411
Number of examples: 3603


In [100]:
pred = classifier.predict(list(test['lemmas']))[0]
pred = [int(label[0][-1]) for label in pred]

accuracy_score(list(y_test), pred)

0.8739938939772411

In [101]:
baseline_6 = [[i, pred[i]] for i in range(len(test))]
baseline_6 = pd.DataFrame(baseline_6, columns=['comment_id', 'toxic'])

baseline_6.to_csv('baseline_6.csv', index=False)